# Table of Contents
1. [start](#start)
2. [pwd](#pwd)
3. [dns-ldap](#dns-ldap)
4. [recon](#recon)
5. [fuzz](#fuzz)
6. [print](#print)

## Begin <a name="start"></a>

In [ ]:
# expected to run with papermill, so gonna do some additional santizing as i can
input_data="10.13.37.14"

In [ ]:
#papermill_description=Setup_target

import netaddr, netifaces, pwn, time,re
from dns import resolver
iplist = [netifaces.ifaddresses(i)[netifaces.AF_INET][0]["addr"] for i in netifaces.interfaces() if netifaces.AF_INET in netifaces.ifaddresses(i)]
print("Local Interfaces:")
for i in iplist: 
    print(i)
ip=""
Host=""
input_data=str(input_data)
print("\nTarget:\n"+input_data)
try:
    try:
        # try to identify if this is a valid ip address
#        print(netaddr.valid_ipv4(input_data))
        if netaddr.valid_ipv4(input_data):
            ip=input_data
        elif netaddr.valid_ipv6(input_data):
            ip=input_data
        else:
            ip=""
#        print(ip)
    except:
        print('not an ip address')
    if ip == "":
        # this case exists to resolve the domain name to provide both ip and host
        try:
            ip=resolver.resolve(input_data, "A").response.answer[0][0].address
            print('ip = '+str(ip))
            Host=str(input_data)
        except:
            print('host lookup failed')
            # this would mean ip = "" and the input data isn't a resolvable name
    else:
        #this case is if ip exists, set host to ip address
        Host=str(input_data)
except:
    print("put proper address")
    exit()
    

## Setup working directory <a name="pwd"></a>

In [ ]:
#papermill_description=SetupWorkingDirectory
import re, requests, ldap3
hostsString=ip+" "+Host
!echo {hostsString} >> /etc/hosts
workingDir="/share/pentests/"+str(Host)
!mkdir {workingDir} 2>/dev/null
print("Changing directory to: ")
%cd {workingDir}

## Check if either DNS or LDAP <a name="dns-ldap"></a>

In [ ]:
#papermill_description=CheckDNS_and_LDAP
# it seemed far quicker to just try it and see what happens, instead of finding if these ports exist first. may change later
try:
    if Host != ip:
        testdns=resolver.Resolver(configure=False)
        testdns.nameservers=[ip]
        testdns.resolve(Host, "a")
    !dnsrecon -r {ip}/32 -n {ip} > self-dns-check
    !dnsrecon -r {ip}/28 -n {ip} >> self-dns-check
    !dnsrecon -r {ip}/24 -n {ip} >> self-dns-check
    !dnsrecon -r 127.0.0.1/24 -n {ip} >> self-dns-check
    !dnsrecon -d {Host} -a >> self-dns-check
    !dnsrecon -d {Host} -D /share/pentests/wordlists/dns.txt >> self-dns-check
    command="""n=`awk -F "\t" '{print $2}' self-dns-check |sort -u|grep -iv localhost|sed /^[[:space:]]*$/d|awk '{print $2}' |sed s/\$/" "/g|tr -d '\\n'`; o=`grep -m1 """+Host+""" /etc/hosts`; e=`whatweb http://"""+Host+""" 2>&1|grep -i error|awk '{print $8}'|strings`; sed "s/$o/$o $n $e/g" -i /etc/hosts"""
    !{command}
except:
    print('not a dns server')
    pass
try:
 server=ldap3.Server(ip, get_info=ldap3.ALL, port=389, use_ssl=False)
 connection=ldap3.Connection(server)
 connection.bind()
 dcinfo=server.info
 print(dcinfo)
except:
    print('likely not an ldap server')
    pass

## Start recon <a name="recon"></a>

In [ ]:
#papermill_description=Nmap_and_whatweb
!nmap -sS -sV -p1- -Pn -n --open -oA {Host}.nmap {ip} 2>/dev/null
result=!grep -h http {Host}.nmap.nmap|grep -i "open"|awk -F "/" '{{print $$1}}'
print("Web ports")
print(str(result))
notwebresults=!grep -h open {Host}.nmap.nmap|grep -iv "http\|scan initiated"|awk -F "/" '{{print $$1}}'
print("Not web ports:")
print(str(notwebresults))
!cat {Host}.nmap.xml |jc --xml|jq . > {Host}-nmap.json
for port in result:
    !whatweb http://{Host}:$port >> whatweb-{Host} 2>/dev/null
    !whatweb https://{Host}:$port >> whatweb-{Host} 2>/dev/null

## Start Fuzz <a name="fuzz"></a>

In [ ]:
#papermill_description=Begin_Fuzzing
def fuzzing():
    for port in result:
        for line in open("/etc/hosts","r").readlines():
            if Host in line:
                for name in set(str(line).replace("\n","").split(" ")):
                    if "." in name:
                        #print(name+":"+port)
                        try:
                            testname='.'.join(name.split('.')[-2:])
                            check=requests.get('http://'+name+":"+port, timeout=20)
                            length=str(len(check.content))
                            if check.status_code: #check if http
                                command="ffuf -s -v -u http://"+name+":"+port+"/FUZZ -w /share/pentests/wordlists/webcontent-top-10000.txt -o ffuf-"+name+"-"+port+"-http-log >/dev/null"
                                output=!{command}
                                #print(output)
                                command="ffuf -s -v -u http://"+name+":"+port+" -H 'Host: FUZZ."+name+"' -w /share/pentests/wordlists/dns-top-10000.txt -o ffuf-"+name+"-"+port+"-dns-log >/dev/null"
                                output=!{command}
                                #print(output)
                                if name != ip: #instead of re-checking that it's an ip, just check to make sure an ip wasn't pulled from hosts (a situation that happens when overwriting it wrong
                                    command="ffuf -s -v -u http://"+name+":"+port+" -H 'Host: FUZZ."+testname+"' -w /share/pentests/wordlists/dns-top-10000.txt -fs "+length+" -o ffuf-"+testname+"-"+port+"-dns-log >/dev/null"
                                    output=!{command}
                                    #print(output)
                            else: #just go ahead and try https, if this fails, it's irrelevant to put into another if clause
                                command="ffuf -s -v -u https://"+name+":"+port+"/FUZZ -w /share/pentests/wordlists/webcontent-top-10000.txt -o ffuf-"+name+"-"+port+"-http-log >/dev/null"
                                output=!{command}
                                #print(output)
                                command="ffuf -s -v -u https://"+name+":"+port+" -H 'Host: FUZZ."+name+"' -w /share/pentests/wordlists/dns-top-10000.txt -o ffuf-"+name+"-"+port+"-dns-log >/dev/null"
                                output=!{command}
                                #print(output)
                                if name != ip: #instead of re-checking that it's an ip, just check to make sure an ip wasn't pulled from hosts (a situation that happens when overwriting it wrong
                                    command="ffuf -s -v -u https://"+name+":"+port+" -H 'Host: FUZZ."+testname+"' -w /share/pentests/wordlists/dns-top-10000.txt -fs "+length+" -o ffuf-"+testname+"-"+port+"-dns-log >/dev/null"
                                    output=!{command}
                                    #print(output)
                        except Exception as e: 
                            #print(e)
                            pass
                break

fuzzing()
command="""n=`awk -F "\t" '{print $2}' self-dns-check |sort -u|grep -iv localhost|sed /^[[:space:]]*$/d|awk '{print $2}' |sed s/"$"/" "/g|tr -d '\\n'`; o=`grep -m1 """+Host+""" /etc/hosts`; e=`cat ffuf-*-dns-log|jq -c '.results[] | {url:.url,host:.host,status:.status,length:.length}|map(.)'|awk -F ',' '!s[$4]++' |awk -F '"' '{print $4}'|sort|uniq |sed s/"$"/" "/g|tr -d '\\n'`; sed "s/$o/$o $n $e/g" -i /etc/hosts; grep """+Host+""" /etc/hosts"""
output=!{command}
data=[]
for i in output:
    for o in i.split(' '):
        data.append(o)
print("found domain results")
for each in set(data):
    print(each)
fuzzing()
time.sleep(2)
command="""for each in `ls ffuf-*`; do cat $each | jq -c '.results[] | {url:.url,status: .status}'; done|sort -u"""
output=!{command}
print("found working urls:")
for i in output:
    if "200" in i or "300" in i or "302" in i or "503" in i or "302" in i:
        re.compile("([:/.A-z?<_&\s=>0-9;-]+)")
        re.findall(str(i))

## Use fuzz results to crawl <a name="print"></a>

In [ ]:
#papermill_description=CrawlUrlList
!cat ffuf-*-log |jq ".results[] |.url, .input.FUZZ, .status"|sed 'N;N;s/\n/ /g' |sed s/'" "'/'\/'/g|sed s/'"'//g|grep -i "200\|301" |awk '{print $1}'|sort -u |docker exec -i kali-0 hakrawler |sort -u -T ./